In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv",
    index_col=0
    )
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


## Saneamento de Dados

### Seleção de colunas & Tratamento dos nulos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336776 entries, 1 to 336776
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       336776 non-null  int64  
 1   month      336776 non-null  int64  
 2   day        336776 non-null  int64  
 3   dep_time   328521 non-null  float64
 4   dep_delay  328521 non-null  float64
 5   arr_time   328063 non-null  float64
 6   arr_delay  327346 non-null  float64
 7   carrier    336776 non-null  object 
 8   tailnum    334264 non-null  object 
 9   flight     336776 non-null  int64  
 10  origin     336776 non-null  object 
 11  dest       336776 non-null  object 
 12  air_time   327346 non-null  float64
 13  distance   336776 non-null  int64  
 14  hour       328521 non-null  float64
 15  minute     328521 non-null  float64
dtypes: float64(7), int64(5), object(4)
memory usage: 43.7+ MB


Critérios :
1. ["arr_time", "dep_time", "carrier", "flight"] não podem ser nulos
2. As observações nulas destes campos devem ser removidos
3. Se fosse para tratar os nulos usar: ex df["dep_time"].fillna("NA")

In [4]:
usecols=["dep_time","arr_time","carrier","flight", "tailnum","air_time","distance", "origin", "dest"]

In [20]:
df_raw = df.loc[
    (~df["arr_time"].isna()) \
    & (~df["dep_time"].isna()) \
    & (~df["carrier"].isna()) \
    & (~df["flight"].isna())
].loc[:, usecols]

In [18]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328063 entries, 1 to 336770
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   dep_time  328063 non-null  float64
 1   arr_time  328063 non-null  float64
 2   carrier   328063 non-null  object 
 3   flight    328063 non-null  int64  
 4   tailnum   328063 non-null  object 
 5   air_time  327346 non-null  float64
 6   distance  328063 non-null  int64  
 7   origin    328063 non-null  object 
 8   dest      328063 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 25.0+ MB


In [21]:
df_raw["air_time"] = df_raw["air_time"].fillna(0)

In [22]:
df_raw.drop_duplicates(inplace=True)

In [23]:
df_raw = df_raw.astype("object")

Perguntas:
1. Qual a vantagem de avaliar os nulos no começo do processo? Podemos pegar erro logo de início
2. Quais os riscos envolvidos? Se estvermos em um pipeline, perdemos a "foto" do dado bruto para uma camada raw. A boa prática em um pipeline seria realizar apenas a transformação para str / objeto

In [32]:
tmp = df.loc[:, usecols].copy()
for col in ["arr_time", "dep_time", "carrier", "flight"]:
    tmp_df = tmp.loc[~df[col].isna()]
    tmp = tmp_df.copy()

In [33]:
tmp.drop_duplicates(inplace=True)

In [35]:
tmp.shape[0] == df_raw.shape[0]

True

### Rename e tipagem

In [36]:
new_columns = ["datetime_partida", "datetime_chegada", "", "id_voo", "id_aeronave","tempo_voo", "distancia", "origem", "destino"]

In [37]:
columns_map = {usecols[i]: new_columns[i] for i in range(len(usecols))}
columns_map

{'dep_time': 'datetime_partida',
 'arr_time': 'datetime_chegada',
 'carrier': 'companhia',
 'flight': 'id_voo',
 'tailnum': 'id_aeronave',
 'air_time': 'tempo_voo',
 'distance': 'distancia',
 'origin': 'origem',
 'dest': 'destino'}

In [38]:
df_work = df_raw.copy()
df_work.rename(columns=columns_map, inplace=True)
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino
1,517.0,830.0,UA,1545,N14228,227.0,1400,EWR,IAH
2,533.0,850.0,UA,1714,N24211,227.0,1416,LGA,IAH
3,542.0,923.0,AA,1141,N619AA,160.0,1089,JFK,MIA
4,544.0,1004.0,B6,725,N804JB,183.0,1576,JFK,BQN
5,554.0,812.0,DL,461,N668DN,116.0,762,LGA,ATL


In [39]:
df_work.dtypes

datetime_partida    object
datetime_chegada    object
companhia           object
id_voo              object
id_aeronave         object
tempo_voo           object
distancia           object
origem              object
destino             object
dtype: object

In [40]:
df_work["tempo_voo"] = df_work.loc[:,"tempo_voo"].astype(float)
df_work["distancia"] = df_work.loc[:,"distancia"].astype(float)
df_work["companhia"] = df_work.loc[:,"companhia"].astype(str)
df_work["id_voo"] = df_work.loc[:,"id_voo"].astype(str)
df_work["id_aeronave"] = df_work.loc[:,"id_aeronave"].astype(str)
df_work["datetime_partida"] = df_work.loc[:,"datetime_partida"].astype(str)
df_work["datetime_chegada"] = df_work.loc[:,"datetime_chegada"].astype(str)
df_work["origem"] = df_work.loc[:,"origem"].astype(str)
df_work["destino"] = df_work.loc[:,"destino"].astype(str)

In [42]:
df_work.dtypes

datetime_partida     object
datetime_chegada     object
companhia            object
id_voo               object
id_aeronave          object
tempo_voo           float64
distancia           float64
origem               object
destino              object
dtype: object

### Tratamento strings

In [43]:
import re

def padroniza_str(obs):
    return re.sub('[^A-Za-z0-9]+', '', obs.upper())

In [44]:
padroniza_str("ahsuw! @ ++  ~ ç 3n!!rrr")

'AHSUW3NRRR'

Guias práticos para regex:
- https://medium.com/@tomstaite1/everything-you-need-to-know-about-regular-expressions-regex-3cbc5b95146
- https://medium.com/xp-inc/regex-um-guia-pratico-para-express%C3%B5es-regulares-1ac5fa4dd39f

In [45]:
df_work["companhia_formatted"] = df_work.loc[:,"companhia"].apply(lambda x: padroniza_str(x))
df_work["id_voo_formatted"] = df_work.loc[:,"id_voo"].apply(lambda x: padroniza_str(x))
df_work["id_aeronave_formatted"] = df_work.loc[:,"id_aeronave"].apply(lambda x: padroniza_str(x))
df_work["origem_formatted"] = df_work.loc[:,"origem"].apply(lambda x: padroniza_str(x))
df_work["destino_formatted"] = df_work.loc[:,"destino"].apply(lambda x: padroniza_str(x))

In [46]:
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted
1,517.0,830.0,UA,1545,N14228,227.0,1400.0,EWR,IAH,UA,1545,N14228,EWR,IAH
2,533.0,850.0,UA,1714,N24211,227.0,1416.0,LGA,IAH,UA,1714,N24211,LGA,IAH
3,542.0,923.0,AA,1141,N619AA,160.0,1089.0,JFK,MIA,AA,1141,N619AA,JFK,MIA
4,544.0,1004.0,B6,725,N804JB,183.0,1576.0,JFK,BQN,B6,725,N804JB,JFK,BQN
5,554.0,812.0,DL,461,N668DN,116.0,762.0,LGA,ATL,DL,461,N668DN,LGA,ATL


### Tratamentos especiais
Neste caso, vamos realizar tratamento das horas

In [47]:
df_work.loc[:,"datetime_partida"] = df_work.loc[:,"datetime_partida"].str.replace('.0', '')
df_work.loc[:,"datetime_chegada"] = df_work.loc[:,"datetime_chegada"].str.replace('.0', '')
df_work["data_voo"] = pd.to_datetime(df[['year', 'month', 'day']]) 

In [48]:
df_work["datetime_partida"].apply(lambda x: len(x)).unique()

array([3, 4, 2, 1], dtype=int64)

In [49]:
def corrige_hora(hr_str,dct_hora = {1:"000?",2:"00?",3:"0?",4:"?"}):
    if hr_str == "2400":
        return "00:00"
    elif (len(hr_str) == 2) & (int(hr_str) <= 12):
        return f"0{hr_str[0]}:{hr_str[1]}0"
    else:
        hora = dct_hora[len(hr_str)].replace("?", hr_str)
        return f"{hora[:2]}:{hora[2:]}"

In [50]:
datetime_partida2 = df_work.loc[:,"datetime_partida"].apply(lambda x: corrige_hora(x))
datetime_chegada2 = df_work.loc[:,"datetime_chegada"].apply(lambda x: corrige_hora(x))

In [51]:
datetime_partida2

1         05:17
2         05:33
3         05:42
4         05:44
5         05:54
          ...  
336766    22:40
336767    22:40
336768    22:41
336769    23:07
336770    23:49
Name: datetime_partida, Length: 328038, dtype: object

In [52]:
pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_partida2)

1        2013-01-01 05:17:00
2        2013-01-01 05:33:00
3        2013-01-01 05:42:00
4        2013-01-01 05:44:00
5        2013-01-01 05:54:00
                 ...        
336766   2013-09-30 22:40:00
336767   2013-09-30 22:40:00
336768   2013-09-30 22:41:00
336769   2013-09-30 23:07:00
336770   2013-09-30 23:49:00
Length: 328038, dtype: datetime64[ns]

In [53]:
df_work['datetime_partida_formatted'] = pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_partida2)
df_work['datetime_chegada_formatted'] = pd.to_datetime(df_work.loc[:,'data_voo'].astype(str) + " " + datetime_chegada2)

In [54]:
df_work.head()

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted,data_voo,datetime_partida_formatted,datetime_chegada_formatted
1,517,830,UA,1545,N14228,227.0,1400.0,EWR,IAH,UA,1545,N14228,EWR,IAH,2013-01-01,2013-01-01 05:17:00,2013-01-01 08:30:00
2,533,850,UA,1714,N24211,227.0,1416.0,LGA,IAH,UA,1714,N24211,LGA,IAH,2013-01-01,2013-01-01 05:33:00,2013-01-01 08:50:00
3,542,923,AA,1141,N619AA,160.0,1089.0,JFK,MIA,AA,1141,N619AA,JFK,MIA,2013-01-01,2013-01-01 05:42:00,2013-01-01 09:23:00
4,544,1004,B6,725,N804JB,183.0,1576.0,JFK,BQN,B6,725,N804JB,JFK,BQN,2013-01-01,2013-01-01 05:44:00,2013-01-01 10:04:00
5,554,812,DL,461,N668DN,116.0,762.0,LGA,ATL,DL,461,N668DN,LGA,ATL,2013-01-01,2013-01-01 05:54:00,2013-01-01 08:12:00


In [55]:
df_work["datetime_chegada_formatted"] = np.where(
    df_work["datetime_partida_formatted"] > df_work["datetime_chegada_formatted"],
    df_work["datetime_chegada_formatted"] + pd.Timedelta(days=1),
    df_work["datetime_chegada_formatted"]
    )

In [56]:
df_work.loc[df_work["datetime_partida_formatted"] > df_work["datetime_chegada_formatted"]]

,datetime_partida,datetime_chegada,companhia,id_voo,id_aeronave,tempo_voo,distancia,origem,destino,companhia_formatted,id_voo_formatted,id_aeronave_formatted,origem_formatted,destino_formatted,data_voo,datetime_partida_formatted,datetime_chegada_formatted


In [57]:
df_dw = df_work[["data_voo",
         "companhia_formatted",
         "id_voo_formatted",
         "id_aeronave_formatted",
         "datetime_partida_formatted",
         "datetime_chegada_formatted",
         "origem_formatted",
         "destino_formatted",
         "tempo_voo",
         "distancia"]].copy()

## Engenharia de Features

- tempo_voo_esperado
- tempo_voo_hr
- atraso (em horas)
- flg_status
- dia_semana
- horario

In [58]:
df_dw["tempo_voo_esperado"] = (df_dw["datetime_chegada_formatted"] - df_dw["datetime_partida_formatted"]) / pd.Timedelta(hours=1)
df_dw["tempo_voo_hr"] = df_dw["tempo_voo"] /60
df_dw["atraso"] = df_dw["tempo_voo_hr"] - df_dw["tempo_voo_esperado"]
df_dw["dia_semana"] = df_dw["data_voo"].dt.day_of_week #0=segunda

In [59]:
def classifica_hora(hra):
    if 0 <= hra < 6: return "MADRUGADA"
    elif 6 <= hra < 12: return "MANHA"
    elif 12 <= hra < 18: return "TARDE"
    else: return "NOITE"

In [60]:
df_dw["horario"] = df_work.loc[:,"datetime_partida_formatted"].dt.hour.apply(lambda x: classifica_hora(x))

In [61]:
df_work.loc[:,"datetime_partida_formatted"].dt.hour.apply(lambda x: classifica_hora(x))

1         MADRUGADA
2         MADRUGADA
3         MADRUGADA
4         MADRUGADA
5         MADRUGADA
            ...    
336766        NOITE
336767        NOITE
336768        NOITE
336769        NOITE
336770        NOITE
Name: datetime_partida_formatted, Length: 328038, dtype: object

In [62]:
df_dw.head()

,data_voo,companhia_formatted,id_voo_formatted,id_aeronave_formatted,datetime_partida_formatted,datetime_chegada_formatted,origem_formatted,destino_formatted,tempo_voo,distancia,tempo_voo_esperado,tempo_voo_hr,atraso,dia_semana,horario
1,2013-01-01,UA,1545,N14228,2013-01-01 05:17:00,2013-01-01 08:30:00,EWR,IAH,227.0,1400.0,3.216667,3.783333,0.566667,1,MADRUGADA
2,2013-01-01,UA,1714,N24211,2013-01-01 05:33:00,2013-01-01 08:50:00,LGA,IAH,227.0,1416.0,3.283333,3.783333,0.500000,1,MADRUGADA
3,2013-01-01,AA,1141,N619AA,2013-01-01 05:42:00,2013-01-01 09:23:00,JFK,MIA,160.0,1089.0,3.683333,2.666667,-1.016667,1,MADRUGADA
4,2013-01-01,B6,725,N804JB,2013-01-01 05:44:00,2013-01-01 10:04:00,JFK,BQN,183.0,1576.0,4.333333,3.050000,-1.283333,1,MADRUGADA
5,2013-01-01,DL,461,N668DN,2013-01-01 05:54:00,2013-01-01 08:12:00,LGA,ATL,116.0,762.0,2.300000,1.933333,-0.366667,1,MADRUGADA


In [63]:
print(
    len(df_dw[df_dw["atraso"] > 5]),
    len(df_dw[df_dw["atraso"] > 6])
    )

447 0


In [66]:
df_dw[df_dw["atraso"]>-1].describe()

,data_voo,datetime_partida_formatted,datetime_chegada_formatted,tempo_voo,distancia,tempo_voo_esperado,tempo_voo_hr,atraso,dia_semana
count,316605,316605,316605,316605.000000,316605.000000,316605.000000,316605.000000,316605.000000,316605.000000
mean,2013-07-03 06:18:01.142748416,2013-07-03 19:57:24.733027328,2013-07-03 22:12:32.416355072,151.593613,1053.883666,2.252134,2.526560,0.274426,2.897500
min,2013-01-01 00:00:00,2013-01-01 05:17:00,2013-01-01 07:02:00,20.000000,80.000000,0.000000,0.333333,-1.000000,0.000000
25%,2013-04-04 00:00:00,2013-04-04 21:23:00,2013-04-05 00:05:00,83.000000,509.000000,1.566667,1.383333,-0.483333,1.000000
50%,2013-07-04 00:00:00,2013-07-04 12:37:00,2013-07-04 14:14:00,130.000000,888.000000,2.266667,2.166667,-0.316667,3.000000
75%,2013-10-01 00:00:00,2013-10-01 21:23:00,2013-10-01 23:19:00,193.000000,1389.000000,2.866667,3.216667,0.650000,5.000000
max,2013-12-31 00:00:00,2013-12-31 23:32:00,2014-01-01 01:20:00,695.000000,4983.000000,7.300000,11.583333,5.750000,6.000000
std,NaN,NaN,NaN,94.564364,741.673085,0.799171,1.576073,1.104304,1.990489


In [67]:
df_filtrada = df_dw[df_dw["atraso"]>-1].copy()

In [68]:
def flg_status(atraso):
    if atraso > 0.5 : return "ATRASO"
    else: return "ONTIME"

In [69]:
df_filtrada["flg_status"] = df_filtrada.loc[:,"atraso"].apply(lambda x: flg_status(x))

In [70]:
df_filtrada["flg_status"].value_counts()

flg_status
ONTIME    214480
ATRASO    102125
Name: count, dtype: int64

In [71]:
df_filtrada.to_csv("nycflights_tratada.csv", index=False)

### Validações

Para validar os nulos é preciso entender a proporção que uma variável possui, e se essa proporção está dentro dos critérios estabelecidos no projeto

In [72]:
len(df.loc[df["tailnum"].isnull()])/len(df)

0.007458963821649999

Para validar as chaves basta saber se o campo(s) chave(s) possuem a mesma quantidade de observações únicos em relação a quantidade de observações da base

In [73]:
len(df_work[['companhia_formatted','datetime_partida_formatted', "id_voo",'datetime_chegada_formatted']].drop_duplicates()) == len(df_work)

True

Para criarmos alertas e/ou registramos em logs o resultado das validações

In [74]:
import logging

logging.basicConfig(filename='flights_pipe.log', level=logging.INFO) #vai criar um arquivo para os logs
logger = logging.getLogger()

In [75]:
logger.info(f'Inicio da execução ; {datetime.datetime.now()}')

In [76]:
if len(df.loc[df["tailnum"].isnull()])/len(df) > 0.00001:
    logger.warning(f"Coluna tailnum possui mais nulos do que o esperado; {datetime.datetime.now()} ")

In [77]:
#Gera uma falha no processo
if len(df.loc[df["tailnum"].isnull()])/len(df) > 0.00001:
    logger.error(f"Coluna tailnum possui mais nulos do que o esperado; {datetime.datetime.now()} ")
    raise Exception("Coluna tailnum possui mais nulos do que o esperado")

Exception: Coluna tailnum possui mais nulos do que o esperado